In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_path = "/content/drive/MyDrive/Data/Peptide-Analysis/train.dat"
test_path = "/content/drive/MyDrive/Data/Peptide-Analysis/test.dat"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [ ]:
train_data.columns = ['String']
train_data = train_data['String'].str.split("\t",n=1 , expand = True )
train_data.columns = ['Class','Peptide']
train_data.head()

,Class,Peptide
0,-1,KADEELFNKLFFGT
1,-1,FLVALHLGTAFALLWYFRKRWCALVRGFFASFGGRRNDDAHMM
2,-1,RDQMRARIADITGVAISRIA
3,-1,RKRLQLLLL
4,-1,PGFCVGEASPLKSPGRRELGHGNLA


In [ ]:
dict_val = {}
for i in range(0,len(train_data)):
    for j in train_data['Peptide'][i]:
        if j not in dict_val:
            dict_val[j] = 1
        else:
            dict_val[j] += 1

print(dict_val)

{'K': 2509, 'A': 3160, 'D': 1880, 'E': 2482, 'L': 3582, 'F': 1505, 'N': 1399, 'G': 2726, 'T': 1905, 'V': 2613, 'H': 896, 'W': 503, 'Y': 1063, 'R': 2322, 'C': 575, 'S': 2312, 'M': 752, 'Q': 1359, 'I': 2337, 'P': 1648, 'X': 4}


In [ ]:
l1 = []
for i in range(0,len(train_data)):
    dict1 = {'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0,'Q':0,'R':0,'S':0,'T':0,'U':0,'V':0,'W':0,'X':0,'Y':0,'Z':0}
    for j in train_data['Peptide'][i]:
        if j not in dict1:
            dict1[j] = 1
        else:
            dict1[j] += 1
    
    str1=[]
    for k in dict1.values():

            str1.append(int(k))
    l1.append(np.asarray(str1))

In [ ]:
train_data['peptide'] = l1
train_data.head(10)

,Class,Peptide,peptide
0,-1,KADEELFNKLFFGT,"[1, 0, 0, 1, 2, 3, 1, 0, 0, 0, 2, 2, 0, 1, 0, ..."
1,-1,FLVALHLGTAFALLWYFRKRWCALVRGFFASFGGRRNDDAHMM,"[6, 0, 1, 2, 0, 6, 4, 2, 0, 0, 1, 6, 2, 1, 0, ..."
2,-1,RDQMRARIADITGVAISRIA,"[4, 0, 0, 2, 0, 0, 1, 0, 4, 0, 0, 0, 1, 0, 0, ..."
3,-1,RKRLQLLLL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0, 0, ..."
4,-1,PGFCVGEASPLKSPGRRELGHGNLA,"[2, 0, 1, 0, 2, 1, 5, 1, 0, 0, 1, 3, 0, 1, 0, ..."
5,-1,TPEEDTMVMKPLPPEPAP,"[1, 0, 0, 1, 3, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, ..."
6,-1,SKDMRVQR,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
7,-1,IAASIRHPLHCT,"[2, 0, 1, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, ..."
8,-1,HPNLNLFAAGHDGGMIVFK,"[2, 0, 0, 1, 0, 2, 3, 2, 1, 0, 1, 2, 1, 2, 0, ..."
9,1,ALWKTLLKKVLKAYSPWTNF,"[2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 4, 0, 1, 0, ..."


In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

X = train_data['peptide']
X = list(X)

y = train_data['Class']
y = y.astype(int)
y = np.where(y < 0, 0, y)

y.reshape(-1,1)

X_ros ,y_ros = ros.fit_resample(X,y)

#print('Original dataset shape', Counter(y))
#print('Resample dataset shape', Counter(y_ros))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size=0.33, random_state=40)

In [ ]:
x_train = np.array(x_train).T
y_train = np.array(y_train)
y_train = np.where(y_train < 0, 0,y_train)
y_train = y_train.reshape(-1,1)

x_test = np.array(x_test).T
y_test = np.array(y_test)
y_test = np.where(y_test < 0, 0,y_test)
y_test = y_test.reshape(-1,1)

In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(26, 1906) (26, 940) (1906, 1) (940, 1)


In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model, Sequential

In [ ]:
np.array(X_ros).shape[1]

26

In [ ]:
Joseph = Sequential()

Joseph.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim= 26))
Joseph.add(Dense(units = 10, activation = 'relu'))
Joseph.add(Dense(units = 5, activation = 'relu'))

Joseph.add(Dense(units = 1, activation = 'sigmoid'))

Joseph.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
print(Joseph.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                270       
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
x_train.shape,y_train.shape

((26, 1906), (1906, 1))

In [ ]:
model = Joseph.fit(x_train.T, y_train, batch_size = 8, epochs = 60)

Epoch 1/60
239/239 [==============================] - 1s 3ms/step - loss: 0.2663 - accuracy: 0.8903
Epoch 2/60
239/239 [==============================] - 0s 2ms/step - loss: 0.1658 - accuracy: 0.9255
Epoch 3/60
239/239 [==============================] - 0s 2ms/step - loss: 0.1482 - accuracy: 0.9381
Epoch 4/60
239/239 [==============================] - 1s 3ms/step - loss: 0.1061 - accuracy: 0.9538
Epoch 5/60
239/239 [==============================] - 1s 2ms/step - loss: 0.1360 - accuracy: 0.9465
Epoch 6/60
239/239 [==============================] - 1s 2ms/step - loss: 0.0981 - accuracy: 0.9622
Epoch 7/60
239/239 [==============================] - 1s 2ms/step - loss: 0.0731 - accuracy: 0.9769
Epoch 8/60
239/239 [==============================] - 1s 2ms/step - loss: 0.0656 - accuracy: 0.9795
Epoch 9/60
239/239 [==============================] - 0s 2ms/step - loss: 0.0549 - accuracy: 0.9853
Epoch 10/60
239/239 [==============================] - 0s 2ms/step - loss: 0.0991 - accuracy: 0.9690

In [ ]:
x_train.T[0]

array([3, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2, 2, 1, 4, 0, 1, 0, 3, 1, 3, 0, 2,
       0, 0, 1, 0])

In [ ]:
score = Joseph.evaluate(x_test.T, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

30/30 [==============================] - 0s 1ms/step - loss: 0.0758 - accuracy: 0.9819
Test Score: 0.0757516399025917
Test Accuracy: 0.9819148778915405
